# Get data and packages

In [1]:
%%capture
%%bash
pip install jupyter_contrib_nbextensions
pip install jupyter_nbextensions_configurator
jupyter contrib nbextension install --user
jupyter nbextensions_configurator enable --user

jupyter nbextension enable codefolding/main
jupyter nbextension enable scratchpad/main
jupyter nbextension enable execute_time/ExecuteTime
jupyter nbextension enable autosavetime/main
pip install -U pip dask numpy fsspec>=0.3.3 tqdm pyarrow


In [2]:
%%capture
!pip install -U pip dask numpy fsspec>=0.3.3 tqdm pyarrow

In [ ]:
!aws s3 cp --recursive s3://xdss-public-datasets/demos/taxi_parquet ../datasets/taxi_parquet

In [55]:
import gc
from src.config import repetitions
import time
import numpy as np
import pandas as pd
import numpy as np
import datetime as dt
import warnings
import time
import gc
import os

name = 'dask'
instance_type = 'mlm52xlarge'
data_path = '../datasets/taxi_parquet/'
results_path = f"../results/{name}_1b_{instance_type}.csv"
benchmarks = {}
print(f"test for {repetitions} repetitions")

test for 1 repetitions


In [61]:
import numpy as np
import pandas as pd
import time 


def get_results(benchmarks, name):
    results = pd.DataFrame.from_dict(benchmarks, orient='index')
    results.columns = [name]
    return results

def persist():
    get_results(benchmarks, name).to_csv(results_path)
    !aws s3 cp  ../results/dask_1b_mlm52xlarge.csv s3://vaex-sagemaker-demo/benchmarks/dask_1b_mlm52xlarge_results.csv 
    
def benchmark(f, df, name, repetitions=1, **kwargs):
    times = []
    for i in range(repetitions):
        start_time = time.time()
        ret = f(df, **kwargs)
        times.append(time.time()-start_time)
    benchmarks[name] = np.mean(times)
    persist()
    print(f"{name} took: {benchmarks[name]}")
    return benchmarks[name]


long_min = -74.05
long_max = -73.75
lat_min = 40.58
lat_max = 40.90

# Benchmark

In [52]:
import dask.dataframe as dd
import numpy as np

# Load data
data = dd.read_parquet(data_path, engine='pyarrow')
print(f"size: {len(data)} with {len(data.columns)} columns")

In [62]:
def read_file_parquet(df=None):
    return dd.read_parquet(data_path, engine='pyarrow')

key = read_file_parquet
f= read_file_parquet
benchmark(f, df=data, name=key, repetitions=repetitions)

upload: ../results/dask_1b_mlm52xlarge.csv to s3://vaex-sagemaker-demo/benchmarks/dask_1b_mlm52xlarge_results.csv


0.7684509754180908

In [63]:
def count(df=None):
    return len(df)

key = 'count'
f = count
benchmark(f, df=data, name=key, repetitions=repetitions)

upload: ../results/dask_1b_mlm52xlarge.csv to s3://vaex-sagemaker-demo/benchmarks/dask_1b_mlm52xlarge_results.csv


48.17900013923645

In [65]:
def mean(df):
    return df.fare_amount.mean().compute()

key = 'mean'
f = mean
benchmark(f, df=data, name=key, repetitions=repetitions)

upload: ../results/dask_1b_mlm52xlarge.csv to s3://vaex-sagemaker-demo/benchmarks/dask_1b_mlm52xlarge_results.csv


13.460501909255981

In [64]:
def standard_deviation(df):
    return df.fare_amount.std().compute()

key = 'standard deviation'
f = standard_deviation
benchmark(f, df=data, name=key, repetitions=repetitions)

upload: ../results/dask_1b_mlm52xlarge.csv to s3://vaex-sagemaker-demo/benchmarks/dask_1b_mlm52xlarge_results.csv


13.961443424224854

To calculate the time when using two columns, we can't return the response since it will get into memroy and break, so we run a mean calculation on it, and then remove the time it took to run the mean.

In [11]:
def mean_of_sum(df):
    return (df.fare_amount + df.trip_distance).mean().compute()

key = 'sum columns mean'
f = mean_of_sum
benchmark(f, df=data, name=key, repetitions=repetitions)
benchmarks['sum columns'] =  benchmarks['sum columns mean'] - benchmarks['mean']

In [14]:
def mean_of_product(df):
    return (df.fare_amount * df.trip_distance).mean().compute()

key = 'product columns mean'
f = mean_of_product
benchmark(f, df=data, name=key, repetitions=repetitions)
benchmarks['product columns'] =  benchmarks['product columns mean'] - benchmarks['mean']

In [15]:
def mean_of_complicated_arithmetic_operation(df):
    theta_1 = df.pickup_longitude
    phi_1 = df.pickup_latitude
    theta_2 = df.dropoff_longitude
    phi_2 = df.dropoff_latitude
    temp = (np.sin((theta_2-theta_1)/2*np.pi/180)**2
           + np.cos(theta_1*np.pi/180)*np.cos(theta_2*np.pi/180) * np.sin((phi_2-phi_1)/2*np.pi/180)**2)
    ret = 2 * np.arctan2(np.sqrt(temp), np.sqrt(1-temp))
    return ret.mean().compute()

key = 'arithmetic operation mean'
f  = mean_of_complicated_arithmetic_operation
benchmark(f, df=data, name=key, repetitions=repetitions)
benchmarks['arithmetic operation'] =  benchmarks['arithmetic operation mean'] - benchmarks['mean']

In [16]:
def value_counts(df):
    return df.fare_amount.value_counts().compute()

key = 'value counts'
f  = value_counts
benchmark(f, df=data, name=key, repetitions=repetitions)

In [ ]:
def groupby_statistics(df):
    return df.groupby(by='passenger_count').agg({'fare_amount': ['mean', 'std'], 
                                               'tip_amount': ['mean', 'std']
                                              }).compute()

key = 'groupby statistics'
f = groupby_statistics
benchmark(f, df=data, name=key, repetitions=repetitions)

In [53]:
def join_count(df, other):
    return len(dd.merge(df, other, left_index=True, right_index=True))

other = groupby_statistics(data)
key = 'join count'
f = join_count
benchmark(f, data, name=key, repetitions=repetitions, other=other)

In [ ]:
# will crash
benchmarks['join'] = -1

## Filtered data

Dask is not build to run on filter data like you would normally, so we will apply the same strategy

In [45]:
print(f"Prepare filtered data and deleted {gc.collect()} MB")

Prepare filtered data and deleted 798 MB


In [47]:
expr_filter = (data.pickup_longitude > long_min)  & (data.pickup_longitude < long_max) & \
                  (data.pickup_latitude > lat_min)    & (data.pickup_latitude < lat_max) & \
                  (data.dropoff_longitude > long_min) & (data.dropoff_longitude < long_max) & \
                  (data.dropoff_latitude > lat_min)   & (data.dropoff_latitude < lat_max)

In [48]:
def filter_data(df):
    return df[expr_filter]

key = 'filter data'
f = filter_data
benchmark(f, data, name=key, repetitions=repetitions)

cleaned 181 mb


In [ ]:
filterd = filter_data(data)

del data
print(f"cleaned {gc.collect()} mb")

In [ ]:
benchmark(filter_data, filterd, name='filterd count', repetitions=repetitions)
benchmark(mean, filterd, name='filterd mean', repetitions=repetitions)
benchmark(standard_deviation, filterd, name='filtered standard deviation', repetitions=repetitions)
benchmark(mean_of_sum, filterd, name ='filtered sum columns mean', repetitions=repetitions)
benchmarks['filtered sum columns'] =  benchmarks['filtered sum columns mean'] - benchmarks['filterd mean']
benchmark(mean_of_product, filterd, name ='filterd product columns mean', repetitions=repetitions)
benchmarks['filterd product columns'] = benchmarks['filterd product columns mean'] - benchmarks['filterd mean']
benchmark(mean_of_complicated_arithmetic_operation, filterd, name='filterd arithmetic operation mean', repetitions=repetitions)
benchmarks['filterd arithmetic operation'] =  benchmarks['filterd arithmetic operation mean'] - benchmarks['filterd mean']
benchmark(value_counts, filterd, name ='filtered value counts', repetitions=repetitions)
benchmark(groupby_statistics, filterd, name='filtered groupby statistics', repetitions=repetitions)
other = groupby_statistics(data)
benchmarks['filtered join'] = -1
benchmarks['filtered join count'] = benchmark(join_count, filterd, repetitions=repetitions, other=other)
print('Done!')